In [ ]:
%pip install --upgrade pip
%pip install pandas
%pip install scikit-learn
%pip install transformers
%pip install torch

In [7]:
import pandas as pd

# Load dataset
data = pd.read_csv("RateMyProfessor_Sample data.csv")

# Extract relevant columns 
data = data[["comments", "star_rating"]]

#dropping all the rows with missing data
data = data.dropna()

import re

def clean_text(text):
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r"\@\w+|\#", "", text)  # Remove mentions and hashtags
    text = re.sub(r"[^\w\s]", "", text)  # Remove special characters
    text = text.lower()  # Lowercase
    return text

data["comments"] = data["comments"].apply(clean_text)




In [8]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)

In [10]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize the data
def tokenize_data(df):
    return tokenizer(
        df["comments"].tolist(),
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt",
    )

train_encodings = tokenize_data(train_data)
val_encodings = tokenize_data(val_data)
test_encodings = tokenize_data(test_data)

/Users/karungopal/.pyenv/versions/3.11.6/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
